# T3 de Inteligência Artificial

Aluno: Leonardo Souza

Professor: Eduardo Bezerra

O objetivo deste trabalho é a implementação do algoritmo Q-Learning pelo método de aproximação linear.

O algoritmo será testado dentro do ambiente Taxi-v3 do OpenGym, para tal, será necessário instalar algumas dependências:

    pip3 install gym==0.17.3



### Testando a criação do ambiente Taxi-v3

In [7]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [8]:
import numpy as np
import torch


In [9]:
state = env.reset()
action = env.action_space.sample() # Seleciona ação a executar
new_state, reward, done, info = env.step(action)
print(env.action_space.n)

print(state)
print(action)
print(new_state, reward, done, info)


def feature_manhattan_distance_taxi_passenger(state, action):
    s_  =  env.P[state][action][0][1]
    

    l, c, p, d = env.unwrapped.decode(s_)
    
    if p >= 4:
        return 0

    p = env.unwrapped.locs[p]
    
    return abs(l - p[0]) + abs(c - p[1])


def feature_manhattan_distance_taxi_destiny(state, action):
    s_  =  env.P[state][action][0][1]
    
    l, c, p, d = env.unwrapped.decode(s_)
    d = env.unwrapped.locs[d]

    return abs(l - d[0]) + abs(c - d[1])


6
201
5
201 -10 False {'prob': 1.0}


In [316]:
# env.s = 352
# state = env.reset()
# print(env)
# print(env.s)
# print(env.P[env.s])
# # print(env.step(5))
# env.s = 0
# print(env)
# print(env.s)
# print(env.P[0])
# # print(env.step(0))


# print(env.action_space)
# print(env.observation_space)
l, c, p, d = env.unwrapped.decode(state)

print(l, c, p, d)

l, c, p, d = env.unwrapped.decode(new_state)

print(l, c, p, d)
print(env.P[0])

neighbors = [ env.P[0][_][0][1] for _ in env.P[0]]
print(neighbors)

2 0 2 0
2 0 2 0
{0: [(1.0, 100, -1, False)], 1: [(1.0, 0, -1, False)], 2: [(1.0, 20, -1, False)], 3: [(1.0, 0, -1, False)], 4: [(1.0, 16, -1, False)], 5: [(1.0, 0, -10, False)]}
[100, 0, 20, 0, 16, 0]


In [79]:
# print(feature_manhattan_distance_taxi_passenger(state, action))
# print(feature_manhattan_distance_taxi_passenger(new_state, action))

# a = np.array([feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny])
b = np.random.rand(a.size)
print(b)
features = [f(state, action) for f in  [feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny]]
# features = np.array([f(state, action) for f in  [feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny]])
print(features)
features = np.array([5, 4])
print(type(features))

print(features * b)
print((features * b).sum())
print( b @ features)

[0.67955667 0.74524331]
[5, 4]
<class 'numpy.ndarray'>
[3.39778335 2.98097324]
6.378756591130507
6.378756591130507


## Implementação do Q-Learning Linear

### Definição dos hyperparâmetros

### Implementação das Classe QLearningLinear

In [38]:
# Class QLearningLinear
class QLearningLinear(object):
    
    def __init__(self, features:list , learning_rate = 0.7, gamma=0.9):
        # cria um numpy array de pesos sendo 1:n, onde n representa o número de features
        self.weights  = np.random.rand(len(features))
        # covnerte a lista de features para um numpy array.
        self.features = features

        self.learning_rate = learning_rate
        self.gamma = gamma
    
    def choose_action(self, state):
        
        max_q = max([(self.Q(state, action), action)  for action in range(6) ])

        return max_q[1]

    def Q(self, state, action):
        """
            Q(S,A) 
        """

        f_values = [f(state, action) for f in  self.features]
        f_values = np.array(f_values)

        return (self.weights  * f_values).sum()
        
    # def 
    def learn(self, state, action, reward, next_state, done, episode):
        '''
            alhpa == learning rate
            gamma == decaiment

            wi <= wi + alpha * [r + gamma * max_Q(s',a') - Q(s,a)]
        '''

        # neighbors = [ env.P[next_state][_][0][1] for _ in env.P[next_state]]
        # max_q = [self.Q(v, None)  for v in neighbors ]
        # max_q = max(max_q)
        # neighbors = [ env.P[next_state][_][0][1] for _ in env.P[next_state]]

        max_q = [self.Q(next_state, action)  for action in range(6) ]
        max_q = max(max_q)

        q = self.Q(state, action)

        self.weights  = self.weights  + self.learning_rate * (reward + self.gamma * max_q - q)

        return self.weights 


In [39]:
qll = QLearningLinear([feature_manhattan_distance_taxi_passenger, feature_manhattan_distance_taxi_destiny], learning_rate=0.2)
state = env.reset()
print(state)

72


In [40]:
# action  = 0
# new_state, reward, done, info = env.step(action)
# print(env.action_space.n)

# print(state)
# print(action)
# print(new_state, reward, done, info)


# print(qll.learn(state, action, reward, new_state, done, 0))
# print(qll.weigths)

def train_fit(agent, env, epochs):
    initial_state = env.reset()
    state = initial_state
    hist = []

    for epoch in range(epochs):
        
        action = agent.choose_action(state)
        
        # transição
        new_state, reward, done, info = env.step(action)
        # print(action)
        weights  = agent.learn(state, action, reward, new_state, done, epoch)

        state = new_state
        
        hist.append([state, action, weights ])
        # episode_rewards.append(reward)
        
        if done == True:
          break
    
    return hist

In [41]:
# Testando o histórico de mudanças

history_states =  train_fit(qll, env, 10)
for i in history_states:
    print(i)
    
print(qll.choose_action(state))


[51, 5, array([-1.99031341, -1.34113318])]
[151, 0, array([-1.28433103, -0.6351508 ])]
[251, 0, array([-0.99796877, -0.34878854])]
[351, 0, array([-0.88172278, -0.23254256])]
[451, 0, array([-0.92495058, -0.27577035])]
[431, 3, array([-1.09542076, -0.44624053])]
[431, 5, array([-3.05566272, -2.40648249])]
[431, 5, array([-4.89829017, -4.24910994])]
[431, 5, array([-6.63035996, -5.98117974])]
[431, 5, array([-8.25850558, -7.60932535])]
0
1
0


In [313]:
# state = env.encode(0, 0, 4, 0)
# print("State id:", state)
# env.s = state

# l, c, p, d = env.unwrapped.decode(s_)
# print(l, c, p, d)
# print(env.unwrapped.locs[d])
# print(env.unwrapped.locs)
# env.render()

State id: 16
0 0 4 0
(0, 0)
[(0, 0), (0, 4), (4, 0), (4, 3)]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


In [328]:
for i in range(10):
    state = env.encode(, 0, 4, 0)
    env.s = state
    env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
|_: | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (South)
